In [ ]:
import json
import random
import io
import ast
from PIL import Image, ImageDraw, ImageFont
from PIL import ImageColor
import xml.etree.ElementTree as ET
additional_colors = [colorname for (colorname, colorcode) in ImageColor.colormap.items()]

def plot_bounding_boxes(img, bounding_boxes, input_width, input_height, save_path=None):

    width, height = img.size
    draw = ImageDraw.Draw(img)

    colors = [
        'red',
        'green',
        'blue',
        'yellow',
        'orange',
        'pink',
        'purple',
        'brown',
        'gray',
        'beige',
        'turquoise',
        'cyan',
        'magenta',
        'lime',
        'navy',
        'maroon',
        'teal',
        'olive',
        'coral',
        'lavender',
        'violet',
        'gold',
        'silver',
    ] + additional_colors

    bounding_boxes = parse_json(bounding_boxes)
    font = ImageFont.load_default()

    try:
        json_output = ast.literal_eval(bounding_boxes)
    except Exception as e:
        end_idx = bounding_boxes.rfind('"}') + len('"}')
        truncated_text = bounding_boxes[:end_idx] + "]"
        json_output = ast.literal_eval(truncated_text)

    for i, bounding_box in enumerate(json_output):
        color = colors[i % len(colors)]
        abs_y1 = int(bounding_box["bbox_2d"][1]/input_height * height)
        abs_x1 = int(bounding_box["bbox_2d"][0]/input_width * width)
        abs_y2 = int(bounding_box["bbox_2d"][3]/input_height * height)
        abs_x2 = int(bounding_box["bbox_2d"][2]/input_width * width)

        if abs_x1 > abs_x2:
            abs_x1, abs_x2 = abs_x2, abs_x1

        if abs_y1 > abs_y2:
            abs_y1, abs_y2 = abs_y2, abs_y1

        draw.rectangle(
            ((abs_x1, abs_y1), (abs_x2, abs_y2)), outline=color, width=4
        )

        if "label" in bounding_box:
            draw.text((abs_x1 + 8, abs_y1 + 6), bounding_box["label"], fill=color, font=font)
            
    img.show()

def parse_json(json_output):
    lines = json_output.splitlines()
    for i, line in enumerate(lines):
        if line == "```json":
            json_output = "\n".join(lines[i+1:])  
            json_output = json_output.split("```")[0]
            break
    return json_output

In [ ]:
import torch
img_out_path = "/root/letractien/Mammo-VLM/out/EMBED/9233433169011334_1.2.826.0.1.3680043.8.498.94415814578689499280115653748407801319.png"
# img_out_path = "/root/letractien/data/nas07/Dataset/Image/EMBED/jpgs/9707944390811804_1.2.826.0.1.3680043.8.498.10949127961531310340472628327650426955.jpg"
input_width, input_height = torch.tensor(640), torch.tensor(640)
response = """
[
    {"bbox_2d": [145, 338, 165, 360], "label": "SC"},
    {"bbox_2d": [379, 358, 409, 388], "label": "SC"},
    {"bbox_2d": [200, 565, 220, 585], "label": "SC"}
]
"""
image = Image.open(img_out_path)
print(image.size)

image.thumbnail([640, 640], Image.Resampling.LANCZOS)
plot_bounding_boxes(image, response, input_width, input_height, img_out_path)